![image](https://github.com/eWaterCycle/ewatercycle/raw/main/docs/examples/logo.png)

# Running Hype model using eWaterCycle package

This notebook shows how to run [Hype](http://www.smhi.net/hype/wiki/doku.php?id=start) model using a demo use-case.

In [1]:
import ewatercycle.models
import ewatercycle.parameter_sets

/home/stefanv/mamba/envs/ewatercycle/lib/python3.10/site-packages/esmvalcore/experimental/_warnings.py:13: UserWarning: 
  Thank you for trying out the new ESMValCore API.
  Note that this API is experimental and may be subject to change.
  More info: https://github.com/ESMValGroup/ESMValCore/issues/498


## Load parameter set

The parameter set `demo.zip` should be downloaded from https://sourceforge.net/projects/hype/files/release_hype_5_6_2/ and unzipped.


In [2]:
parameter_set_dir = "<path where demo.zip was extracted to>"
parameter_set_dir = "/home/stefanv/git/eWaterCycle/ewatercycle/docs/examples/hype-run/demo"
parameter_set = ewatercycle.parameter_sets.ParameterSet(
    name="hype_demo",
    directory=parameter_set_dir,
    config=parameter_set_dir + "/info.txt",
    target_model="hype",
)

## Setting up the model

Note that the model version and the parameterset versions should be compatible.

In [3]:
ewatercycle.models.Hype.available_versions

('feb2021',)

In [4]:
model = ewatercycle.models.Hype(version="feb2021", parameter_set=parameter_set)

eWaterCycle exposes a selected set of configurable parameters. These can be modified in the `setup()` method.

In [5]:
model.parameters

[('start_time', '1961-01-01T00:00:00Z'),
 ('end_time', '1963-12-31T00:00:00Z'),
 ('crit_time', '1962-01-01T00:00:00Z')]

Calling `setup()` will start up a docker or singularity container. Be careful with calling it multiple times!

In [6]:
cfg_file, cfg_dir = model.setup()
cfg_file, cfg_dir

SingularityVersionException: Wrong version (1.0.2) of singularity found, require version 3.6.0

Note that the parameters have been changed. A new config file which incorporates these updated parameters has been generated as well. If you want to see or modify any additional model settings, you can acces this file directly. When you're ready, pass the path to the config file to `initialize()`.

In [7]:
model.initialize(cfg_file)

## Running the model

Simply running the model from start to end is straightforward. At each time step we can retrieve information from the model.

In [15]:
while model.time < model.end_time:
    print(model.time, end="\r")
    model.update()

In [14]:
model.bmi.get_time_units()

'hours since start of simulation'

## Interacting with the model

Hype model exposes many variables.

In [18]:
model.output_var_names

('comp outflow olake',
 'rec outflow olake',
 'air temperature',
 'corrected air temper',
 'precipitation',
 'corr precipitation',
 'subbasin evaporation',
 'computed runoff',
 'computed soil water')

Hype is a lumped sub-basin model so there are values per subbasin. The outflow of all sub-basins can be requested with:

In [22]:
da = model.get_value("comp outflow olake")
da

array([0.03541804])

The model has some info about the sub-basins:

In [23]:
[
    model.bmi.get_grid_size(1),
    model.bmi.get_grid_rank(1),
    model.bmi.get_grid_type(1),
    model.bmi.get_grid_shape(1),
    model.bmi.get_grid_x(1),
    model.bmi.get_grid_y(1),
]

[1, 1, 'unstructured', (1,), array([0.]), array([0.])]

## Cleaning up

Models usually perform some "wrap up tasks" at the end of a model run, such as writing the last outputs to disk and releasing memory. In the case of eWaterCycle, another important teardown task is destroying the docker or singularity container in which the model was running. This can free up a lot of resources on your system. Therefore it is good practice to always call `finalize()` when you're done with an experiment.

In [24]:
model.finalize()